# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [36]:
from selenium.webdriver.chrome import service
from selenium import webdriver
from bs4 import BeautifulSoup

import time
import re


# using opera driver
webDriverServer = service.Service('./Data/operadriver_win64/operadriver.exe')
webDriverServer.start()

capabilities = {
    'operaOptions': {
        'binary': 'C:/Program Files (x86)/Opera/launcher.exe'
    }
}

browser = webdriver.Remote(webDriverServer.service_url, webdriver.DesiredCapabilities.OPERA)

browser.get("https://www.ettoday.net/news/news-list.htm")

# --- scroll to bottom ---
# get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

# load infinite page
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # pause to load page
    time.sleep(1)
    
    # compare new scroll height with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
# --- get page data ---
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html5lib')

In [37]:
# --- extract today's news ---
regex = time.strftime("%Y/%m/%d", time.localtime())
pattern = re.compile(regex)
for news in soup.find(class_ = "part_list_2").find_all("h3"):
    result = re.search(pattern, news.find(class_ = "date").string)
    if result:
        print( " ".join(news.find_all(text = True)) )

2020/07/06 18:06 大陸 陸外長王毅與印度國安顧問通話　達成四項共識「盡快令一線部隊脫離接觸」
2020/07/06 18:05 財經 王道銀信用卡換裝「戽斗星球」超吸睛　網購最高回饋5%！
2020/07/06 18:05 ET車雲 Smart法國工廠關閉產線移轉大陸　1,600名員工何去何從？
2020/07/06 18:03 社會 16歲少年親割斷母親頸動脈！舅舅曝「不是第一次」：他早就成魔了
2020/07/06 18:03 國際 女國手宿舍輕生亡！部長致歉...教練、隊長「否認施虐」落淚：感到心痛
2020/07/06 18:02 寵物動物 我可以摸嗎？　阿嬤70年來「第一次抱貓」...嬰兒姿勢托著笑開懷！
2020/07/06 18:01 大陸 觸港版國安法首件案例開庭　男持「港獨」旗被控兩宗罪
2020/07/06 18:00 社會 LIVE／PF32「假Coser妹」現身了！穿連身裙大方揮手：辛苦囉～靠么擋路
2020/07/06 17:59 健康 中藥治療新冠有成效！　三總把「氣切重症患」從鬼門關拉回來
2020/07/06 17:58 生活 女同事頭髮臭翻「5天洗一次」：沒味道啊！他閉氣想吐：根本屍臭味
2020/07/06 17:57 社會 遭控性侵衛生所女員工　執行長再發聲：我也是受害者！絕對沒有強迫
2020/07/06 17:56 政治 馬英九罕見室內全程「戴墨鏡」　原因曝光
2020/07/06 17:56 大陸 丈夫意外去世！妻哀求「繼續胚胎移植」遭拒...醫：她是單身婦女
2020/07/06 17:55 生活 台博館鐵道部園區7/7開幕！館內有莒光號可搭　窗外還會有光影、聲音
2020/07/06 17:55 大陸 陸外交部：海警發現日本漁船非法進入釣魚台海域
2020/07/06 17:54 社會 鮮肉男要好好保護自己！前男友裸照威脅拿300萬「再要750萬」　他崩潰報警
2020/07/06 17:52 生活 「2300萬動態模型」重現第3代火車站場景！台博館鐵道部園區7/7開館
2020/07/06 17:51 消費 【廣編】iRent推「多元運具，優惠加倍」　最高送汽車270分鐘、機車90分鐘
2020/07/06 17:51 社會 偵查佐暗槓詐欺案35萬「家裡有急用」　貪汙罪起訴檢方請求減刑
2020/07

## 2. 取出現在時間兩小時內的新聞

In [38]:
# --- extract 2 hours ago's news ---
two_hours_ago = time.time() - 2 * 60 * 60
for news in soup.find(class_ = "part_list_2").find_all("h3"):
    news_date = news.find(class_ = "date").string
    news_date_tick = time.mktime(time.strptime(news_date,"%Y/%m/%d %H:%M"))
    if news_date_tick < two_hours_ago:
        break
    print( " ".join(news.find_all(text = True)) )

2020/07/06 18:06 大陸 陸外長王毅與印度國安顧問通話　達成四項共識「盡快令一線部隊脫離接觸」
2020/07/06 18:05 財經 王道銀信用卡換裝「戽斗星球」超吸睛　網購最高回饋5%！
2020/07/06 18:05 ET車雲 Smart法國工廠關閉產線移轉大陸　1,600名員工何去何從？
2020/07/06 18:03 社會 16歲少年親割斷母親頸動脈！舅舅曝「不是第一次」：他早就成魔了
2020/07/06 18:03 國際 女國手宿舍輕生亡！部長致歉...教練、隊長「否認施虐」落淚：感到心痛
2020/07/06 18:02 寵物動物 我可以摸嗎？　阿嬤70年來「第一次抱貓」...嬰兒姿勢托著笑開懷！
2020/07/06 18:01 大陸 觸港版國安法首件案例開庭　男持「港獨」旗被控兩宗罪
2020/07/06 18:00 社會 LIVE／PF32「假Coser妹」現身了！穿連身裙大方揮手：辛苦囉～靠么擋路
2020/07/06 17:59 健康 中藥治療新冠有成效！　三總把「氣切重症患」從鬼門關拉回來
2020/07/06 17:58 生活 女同事頭髮臭翻「5天洗一次」：沒味道啊！他閉氣想吐：根本屍臭味
2020/07/06 17:57 社會 遭控性侵衛生所女員工　執行長再發聲：我也是受害者！絕對沒有強迫
2020/07/06 17:56 政治 馬英九罕見室內全程「戴墨鏡」　原因曝光
2020/07/06 17:56 大陸 丈夫意外去世！妻哀求「繼續胚胎移植」遭拒...醫：她是單身婦女
2020/07/06 17:55 生活 台博館鐵道部園區7/7開幕！館內有莒光號可搭　窗外還會有光影、聲音
2020/07/06 17:55 大陸 陸外交部：海警發現日本漁船非法進入釣魚台海域
2020/07/06 17:54 社會 鮮肉男要好好保護自己！前男友裸照威脅拿300萬「再要750萬」　他崩潰報警
2020/07/06 17:52 生活 「2300萬動態模型」重現第3代火車站場景！台博館鐵道部園區7/7開館
2020/07/06 17:51 消費 【廣編】iRent推「多元運具，優惠加倍」　最高送汽車270分鐘、機車90分鐘
2020/07/06 17:51 社會 偵查佐暗槓詐欺案35萬「家裡有急用」　貪汙罪起訴檢方請求減刑
2020/07

## 3. 根據範例，取出三天前下午三點到五點的新聞

In [39]:
from selenium.webdriver.support.ui import Select

selectDate = Select(browser.find_element_by_id("selD"))

today_date = time.localtime().tm_mday
selectDate.select_by_value( str(today_date - 3) )

browser.find_element_by_id("button").click()

In [40]:
# --- scroll to bottom ---
# get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

# load infinite page
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # pause to load page
    time.sleep(1)
    
    # compare new scroll height with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# --- get page data ---
html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")

In [41]:
# --- extract news from 3 p.m. to 5 p.m. 3 days ago ---
regex = time.strftime("%Y/%m", time.localtime()) + "/[0]*" + str(today_date - 3) + "\s(?:(?:13|14):\d+|15:00)"
pattern = re.compile(regex)
for news in soup.find(class_ = "part_list_2").find_all("h3"):
    result = re.search(pattern, news.find(class_ = "date").string)
    if result:
        print( " ".join(news.find_all(text = True)) )

2020/07/03 15:00 生活 沒戴口罩進不去「險錯過手術時間」！　陌生女孩一舉動暖炸：我這裡有沒關係
2020/07/03 15:00 旅遊 晚上10點才出沒！台中深夜限定豆花　口感超嫩加綠豆更過癮
2020/07/03 14:59 影劇 衝立院20hrs被打到「皮開肉綻」！　藍委上節目公開驗傷…主持人一看傷口笑出來
2020/07/03 14:59 生活 8月後將有33名男同學！北市「靜修女中」今說掰掰　國中男生可直升了
2020/07/03 14:59 國際 海邊拍婚紗照被恐怖巨浪捲走　「白紗泡水超重」新人差點變浮屍
2020/07/03 14:59 財經 高盛: 原油需求在 2022 年前有望恢復到疫情前水平
2020/07/03 14:58 影劇 珉娥揭「在AOA被霸凌10年」父喪被關衣櫃、餵安眠藥　「韓網全猜她」智珉發全黑圖嗆回
2020/07/03 14:58 影劇 劉亞仁1.4億「3層樓透天豪宅」根本美術館！　玻璃紅酒屋＋小公園種菜…他走到腰痠
2020/07/03 14:57 生活 影／時事、科學應用多！指考物理考蘇花改測速　補教師：複雜到老師都算錯
2020/07/03 14:57 社會 漢光演習拖靶船台中海域翻覆！　2工人落海漂流20分鐘救起
2020/07/03 14:54 政治 總統官邸清秀憲兵姐妹花曝光！曾任柔道代表隊...網全戀愛了：想回役
2020/07/03 14:52 地方 屏東市長林恊松因案遭羈押　縣議會前議長程清水「第二度代理」
2020/07/03 14:49 大陸 影／貴州大山傳「嗚」詭異聲...居民聚集狂拍：是龍的呻吟！當局緊急調查
2020/07/03 14:49 社會 墾丁草原驚現「黃土拉鍊」！遊客爽飆沙灘車被開罰15萬　墾管處：盼牛車替代
2020/07/03 14:44 財經 暑期工讀勞動權益要注意！　勞動部：將展開1400場勞動專案檢查
2020/07/03 14:42 影劇 不是焦凡凡！婁峻碩「擁絕美新女友」　旗津看夜景照片曝光
2020/07/03 14:42 生活 好心載他看醫生！老伯車上閒聊「你在哪上班」　聽完秒變臉：別隨便載人
2020/07/03 14:42 軍武 譚兵讀武／節目正式改版！盧溝橋的石獅子怎麼數不完？　今晚間8時首播解密
2020/07/03 14:39 生活 影／美國傳買斷未